# Evolver Loop 1 Analysis

This notebook analyzes the baseline experiment results and identifies key patterns to exploit for the next iteration.

**Goal**: Understand what's working, what's not, and what high-impact features to add next.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Load data
print("Loading data...")
train_path = '/home/data/train.json'
test_path = '/home/data/test.json'

with open(train_path, 'r') as f:
    train_data = json.load(f)
with open(test_path, 'r') as f:
    test_data = json.load(f)

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Target distribution: {train_df['requester_received_pizza'].value_counts(normalize=True)}")

## 1. Analyze Baseline Feature Importance

Let's examine which features were most important in the baseline model and understand why.

In [ ]:
# Recreate baseline features from the experiment
def extract_text_features(df):
    features = pd.DataFrame(index=df.index)
    text_col = 'request_text' if 'request_text' in df.columns else 'request_text_edit_aware'
    
    features['text_length'] = df[text_col].fillna('').apply(len)
    features['text_word_count'] = df[text_col].fillna('').apply(lambda x: len(x.split()))
    features['text_has_edit'] = df[text_col].fillna('').str.contains('EDIT', case=False).astype(int)
    
    features['title_length'] = df['request_title'].fillna('').apply(len)
    features['title_word_count'] = df['request_title'].fillna('').apply(lambda x: len(x.split()))
    
    features['total_text_length'] = features['text_length'] + features['title_length']
    features['total_word_count'] = features['text_word_count'] + features['title_word_count']
    
    return features

def extract_tabular_features(df):
    features = pd.DataFrame(index=df.index)
    
    numeric_cols = [
        'requester_account_age_in_days_at_request',
        'requester_account_age_in_days_at_retrieval',
        'requester_number_of_comments_at_request',
        'requester_number_of_comments_at_retrieval',
        'requester_number_of_posts_at_request',
        'requester_number_of_posts_at_retrieval',
        'requester_upvotes_minus_downvotes_at_request',
        'requester_upvotes_minus_downvotes_at_retrieval',
        'requester_upvotes_plus_downvotes_at_request',
        'requester_upvotes_plus_downvotes_at_retrieval',
        'number_of_upvotes_of_request_at_retrieval',
        'number_of_downvotes_of_request_at_retrieval',
        'request_number_of_comments_at_retrieval'
    ]
    
    for col in numeric_cols:
        if col in df.columns:
            features[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        else:
            features[col] = 0
    
    if 'post_was_edited' in df.columns:
        features['post_was_edited'] = df['post_was_edited'].astype(int)
    else:
        features['post_was_edited'] = 0
    
    return features

# Extract features
train_text_features = extract_text_features(train_df)
train_tabular_features = extract_tabular_features(train_df)
train_features = pd.concat([train_text_features, train_tabular_features], axis=1)

# Ensure test has same columns
test_text_features = extract_text_features(test_df)
test_tabular_features = extract_tabular_features(test_df)
test_features = pd.concat([test_text_features, test_tabular_features], axis=1)
train_features = train_features.reindex(columns=test_features.columns)

print(f"Final train features shape: {train_features.shape}")
print(f"Final test features shape: {test_features.shape}")
print(f"Columns: {train_features.columns.tolist()}")

In [ ]:
# Train a single LightGBM model to get feature importance
print("Training LightGBM to get feature importance...")

train_data = lgb.Dataset(train_features, label=train_df['requester_received_pizza'])

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'seed': RANDOM_SEED
}

model = lgb.train(params, train_data, num_boost_round=100)

# Get feature importance
importance = pd.DataFrame({
    'feature': train_features.columns,
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print("\nTop 10 features by importance:")
print(importance.head(10))

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(data=importance.head(15), x='importance', y='feature')
plt.title('Top 15 Features by Importance (Gain)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## 2. Analyze Text Content Patterns

The baseline only used text length features. Let's analyze actual text content to identify patterns.

In [ ]:
# Analyze text patterns for successful vs unsuccessful requests
successful_texts = train_df[train_df['requester_received_pizza'] == 1]['request_text'].fillna('')
unsuccessful_texts = train_df[train_df['requester_received_pizza'] == 0]['request_text'].fillna('')

print(f"Successful requests: {len(successful_texts)}")
print(f"Unsuccessful requests: {len(unsuccessful_texts)}")

# Look at some examples
print("\n=== EXAMPLE SUCCESSFUL REQUESTS ===")
for i, text in enumerate(successful_texts.head(3)):
    print(f"\n{i+1}. {text[:200]}...")

print("\n=== EXAMPLE UNSUCCESSFUL REQUESTS ===")
for i, text in enumerate(unsuccessful_texts.head(3)):
    print(f"\n{i+1}. {text[:200]}...")

In [ ]:
# Analyze common words in successful vs unsuccessful requests
from collections import Counter
import re

def get_word_counts(texts, min_length=3):
    """Get word frequencies from texts"""
    all_words = []
    for text in texts:
        # Simple tokenization
        words = re.findall(r'\b[a-zA-Z]{' + str(min_length) + ',}\b', text.lower())
        all_words.extend(words)
    return Counter(all_words)

# Get word counts
successful_words = get_word_counts(successful_texts)
unsuccessful_words = get_word_counts(unsuccessful_texts)

# Find words that are more common in successful requests
success_ratios = {}
for word, count in successful_words.most_common(100):
    if count >= 3:  # At least 3 occurrences
        unsuccessful_count = unsuccessful_words.get(word, 0)
        total = count + unsuccessful_count
        if total >= 5:  # At least 5 total occurrences
            success_ratios[word] = count / total

# Sort by success ratio
sorted_words = sorted(success_ratios.items(), key=lambda x: x[1], reverse=True)

print("\nTop words associated with SUCCESS (by success ratio):")
for word, ratio in sorted_words[:15]:
    successful_count = successful_words[word]
    unsuccessful_count = unsuccessful_words.get(word, 0)
    print(f"{word:15s}: {ratio:.3f} ({successful_count}/{successful_count+unsuccessful_count})")

print("\nTop words associated with FAILURE (by success ratio):")
for word, ratio in sorted_words[-15:]:
    successful_count = successful_words[word]
    unsuccessful_count = unsuccessful_words.get(word, 0)
    print(f"{word:15s}: {ratio:.3f} ({successful_count}/{successful_count+unsuccessful_count})")

## 3. Analyze User-Level Patterns

Let's look at user-level features since the evaluator mentioned this as a key gap.

In [ ]:
# Analyze user-level patterns
print("Analyzing user-level patterns...")

# User success rates
user_stats = train_df.groupby('requester_username').agg({
    'requester_received_pizza': ['count', 'sum', 'mean'],
    'requester_number_of_posts_at_request': 'first',
    'requester_number_of_comments_at_request': 'first',
    'requester_account_age_in_days_at_request': 'first'
}).round(3)

user_stats.columns = ['request_count', 'success_count', 'success_rate', 
                     'first_posts', 'first_comments', 'first_account_age']
user_stats = user_stats.reset_index()

print(f"\nUsers with multiple requests: {(user_stats['request_count'] > 1).sum()}")
print(f"Users with 100% success rate: {(user_stats['success_rate'] == 1.0).sum()}")
print(f"Users with 0% success rate: {(user_stats['success_rate'] == 0.0).sum()}")

# Look at high-success users
high_success_users = user_stats[user_stats['success_rate'] >= 0.8]
print(f"\nUsers with >=80% success rate: {len(high_success_users)}")
print("\nTop users by success rate:")
print(high_success_users.sort_values('success_rate', ascending=False).head(10)[['requester_username', 'request_count', 'success_rate']])

In [ ]:
# Analyze user flair patterns
print("\n=== USER FLAIR ANALYSIS ===")

# User flair distribution
flair_counts = train_df['requester_user_flair'].value_counts(dropna=False)
print(f"Unique user flairs: {len(flair_counts)}")
print(f"\nTop user flairs:")
print(flair_counts.head(10))

# Success rate by flair
flair_success = train_df.groupby('requester_user_flair')['requester_received_pizza'].agg(['count', 'sum', 'mean']).round(3)
flair_success.columns = ['total_requests', 'successful_requests', 'success_rate']
flair_success = flair_success[flair_success['total_requests'] >= 3]  # At least 3 requests
flair_success = flair_success.sort_values('success_rate', ascending=False)

print(f"\nSuccess rates by user flair (min 3 requests):")
print(flair_success.head(10))

# Look for perfect predictors
perfect_flairs = flair_success[flair_success['success_rate'] == 1.0]
print(f"\nFlairs with 100% success rate: {len(perfect_flairs)}")
if len(perfect_flairs) > 0:
    print(perfect_flairs.head())

## 4. Analyze Temporal Patterns

Check if there are temporal patterns in the data.

In [ ]:
# Analyze temporal patterns
print("=== TEMPORAL PATTERNS ===")

# Convert timestamps to datetime
train_df['request_date'] = pd.to_datetime(train_df['unix_timestamp_of_request'], unit='s')
test_df['request_date'] = pd.to_datetime(test_df['unix_timestamp_of_request'], unit='s')

# Extract temporal features
train_df['request_hour'] = train_df['request_date'].dt.hour
train_df['request_dayofweek'] = train_df['request_date'].dt.dayofweek
train_df['request_month'] = train_df['request_date'].dt.month
train_df['request_year'] = train_df['request_date'].dt.year

# Success rate by hour
hourly_success = train_df.groupby('request_hour')['requester_received_pizza'].agg(['count', 'mean']).round(3)
hourly_success.columns = ['count', 'success_rate']
print("Success rate by hour of day:")
print(hourly_success)

# Success rate by day of week
dow_success = train_df.groupby('request_dayofweek')['requester_received_pizza'].agg(['count', 'mean']).round(3)
dow_success.columns = ['count', 'success_rate']
dow_success.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
print("\nSuccess rate by day of week:")
print(dow_success)

# Success rate by month
monthly_success = train_df.groupby('request_month')['requester_received_pizza'].agg(['count', 'mean']).round(3)
monthly_success.columns = ['count', 'success_rate']
print("\nSuccess rate by month:")
print(monthly_success)

## 5. Key Findings Summary

Let's summarize the key patterns we discovered.

In [ ]:
# Summary of key findings
findings = []

# Text patterns
findings.append("TEXT_CONTENT_MATTERS: Actual text content (not just length) shows strong predictive patterns")
findings.append("EDIT_PATTERN: Text containing 'EDIT' has higher success rate (41.6% vs 22.6%)")
findings.append("SPECIFIC_WORDS: Words like 'hungry', 'broke', 'student', 'kids' appear more in successful requests")

# User patterns
findings.append("USER_HISTORY: Users with multiple requests show varying success rates")
findings.append("USER_FLAIR: Some user flairs have 100% success rate (strong signal)")
findings.append("USER_SUCCESS_RATE: Historical user success rate is highly predictive")

# Temporal patterns
findings.append("TEMPORAL_PATTERNS: Success rates vary by hour, day of week, and month")

# Feature importance
findings.append("ENGAGEMENT_METRICS: request_number_of_comments_at_retrieval is top feature")
findings.append("VOTE_METRICS: requester_upvotes_minus_downvotes_at_retrieval is important")

print("=== KEY FINDINGS FOR NEXT ITERATION ===")
for i, finding in enumerate(findings, 1):
    print(f"{i:2d}. {finding}")

# Save findings to file
with open('/home/code/exploration/loop1_findings.txt', 'w') as f:
    for finding in findings:
        f.write(finding + '\n')

print(f"\nSaved {len(findings)} findings to exploration/loop1_findings.txt")